# Human cognitive prediction model

## Imports

In [1]:
import pandas as pd
import numpy as np

# charts
import plotly.express as px

# sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

# Custom python scripts
import sys
sys.path.append("../..")

## Analysis

In [2]:
original_df = pd.read_csv("../data/human_cognitive_performance.csv")
original_df.head()

,User_ID,Age,Gender,Sleep_Duration,Stress_Level,Diet_Type,Daily_Screen_Time,Exercise_Frequency,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Cognitive_Score,AI_Predicted_Score
0,U1,57,Female,6.5,3,Non-Vegetarian,6.5,Medium,41,583.33,65,36.71,39.77
1,U2,39,Female,7.6,9,Non-Vegetarian,10.8,High,214,368.24,58,54.35,57.68
2,U3,26,Male,8.2,6,Vegetarian,5.7,Low,429,445.21,49,32.57,29.54
3,U4,32,Male,7.8,9,Vegetarian,8.3,Low,464,286.33,94,70.15,74.59
4,U5,50,Male,9.7,2,Non-Vegetarian,11.3,Medium,365,237.65,62,87.54,91.78


In [3]:
df = original_df.drop(columns=["User_ID", "AI_Predicted_Score"])
df.head()

,Age,Gender,Sleep_Duration,Stress_Level,Diet_Type,Daily_Screen_Time,Exercise_Frequency,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Cognitive_Score
0,57,Female,6.5,3,Non-Vegetarian,6.5,Medium,41,583.33,65,36.71
1,39,Female,7.6,9,Non-Vegetarian,10.8,High,214,368.24,58,54.35
2,26,Male,8.2,6,Vegetarian,5.7,Low,429,445.21,49,32.57
3,32,Male,7.8,9,Vegetarian,8.3,Low,464,286.33,94,70.15
4,50,Male,9.7,2,Non-Vegetarian,11.3,Medium,365,237.65,62,87.54


In [4]:
print(f"Rows: {df.shape[0]}")
print(f"Columns: {df.shape[1]}")

Rows: 80000
Columns: 11


In [5]:
numerical_columns = df.select_dtypes(include=["number"]).columns.tolist()
categorical_columns = df.select_dtypes(include=["object", "category"]).columns.tolist()

print("numerical_columns", numerical_columns)
print("categorical_columns", categorical_columns)

numerical_columns ['Age', 'Sleep_Duration', 'Stress_Level', 'Daily_Screen_Time', 'Caffeine_Intake', 'Reaction_Time', 'Memory_Test_Score', 'Cognitive_Score']
categorical_columns ['Gender', 'Diet_Type', 'Exercise_Frequency']


In [6]:
# Checking null values
df.isna().sum()

Age                   0
Gender                0
Sleep_Duration        0
Stress_Level          0
Diet_Type             0
Daily_Screen_Time     0
Exercise_Frequency    0
Caffeine_Intake       0
Reaction_Time         0
Memory_Test_Score     0
Cognitive_Score       0
dtype: int64

In [7]:
df.describe()

,Age,Sleep_Duration,Stress_Level,Daily_Screen_Time,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Cognitive_Score
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,38.525525,7.005333,5.493762,6.504646,248.988213,399.973579,69.498350,58.172395
std,12.101876,1.734435,2.865308,3.167072,144.541990,115.369329,17.305659,23.058522
min,18.000000,4.000000,1.000000,1.000000,0.000000,200.000000,40.000000,0.000000
25%,28.000000,5.500000,3.000000,3.800000,123.000000,300.150000,55.000000,40.910000
50%,39.000000,7.000000,5.000000,6.500000,249.000000,400.360000,70.000000,58.360000
75%,49.000000,8.500000,8.000000,9.200000,375.000000,499.250000,85.000000,75.830000
max,59.000000,10.000000,10.000000,12.000000,499.000000,599.990000,99.000000,100.000000


## Exploratory data analysis

In [8]:
exercise_counts = df["Exercise_Frequency"].value_counts().reset_index()
exercise_counts.columns = ["Exercise_Frequency", "Count"]

fig = px.bar(
    exercise_counts, x="Exercise_Frequency", y="Count", title="Exercise Frequency"
)
fig.show()

In [9]:
gender = df["Gender"].value_counts().reset_index()

fig = px.bar(gender, x="Gender", y="count", title="Gender")
fig.show()

In [10]:
diet_type = df["Diet_Type"].value_counts().reset_index()

fig = px.bar(diet_type, x="Diet_Type", y="count", title="Diet Type distribution")
fig.show()

In [11]:
age_count = df["Age"].value_counts().reset_index()

fig = px.area(age_count, x="Age", y="count", title="Age distribution")

fig.show()

In [12]:
sleep_duration_count = df["Sleep_Duration"].value_counts().reset_index()

fig = px.area(
    sleep_duration_count,
    x="Sleep_Duration",
    y="count",
    title="Sleep Duration distribution",
)
fig.show()

In [13]:
numeric_df = df.select_dtypes(include=["number"])
correlation_matrix = numeric_df.corr()

fig = px.imshow(
    correlation_matrix,
    text_auto=".2f",
    color_continuous_scale="RdBu_r",
    title="Correlation Matrix",
)
fig.update_layout(width=800, height=700)
fig.show()

## Models

### Linear Regression

In [14]:
df_linear = df.copy()
df_linear.head()

,Age,Gender,Sleep_Duration,Stress_Level,Diet_Type,Daily_Screen_Time,Exercise_Frequency,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Cognitive_Score
0,57,Female,6.5,3,Non-Vegetarian,6.5,Medium,41,583.33,65,36.71
1,39,Female,7.6,9,Non-Vegetarian,10.8,High,214,368.24,58,54.35
2,26,Male,8.2,6,Vegetarian,5.7,Low,429,445.21,49,32.57
3,32,Male,7.8,9,Vegetarian,8.3,Low,464,286.33,94,70.15
4,50,Male,9.7,2,Non-Vegetarian,11.3,Medium,365,237.65,62,87.54


In [15]:
predict_column = 'Cognitive_Score'

df_encoded = pd.get_dummies(df_linear, columns=categorical_columns, drop_first=True)
df_encoded.head()

,Age,Sleep_Duration,Stress_Level,Daily_Screen_Time,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Cognitive_Score,Gender_Male,Gender_Other,Diet_Type_Vegan,Diet_Type_Vegetarian,Exercise_Frequency_Low,Exercise_Frequency_Medium
0,57,6.5,3,6.5,41,583.33,65,36.71,False,False,False,False,False,True
1,39,7.6,9,10.8,214,368.24,58,54.35,False,False,False,False,False,False
2,26,8.2,6,5.7,429,445.21,49,32.57,True,False,False,True,True,False
3,32,7.8,9,8.3,464,286.33,94,70.15,True,False,False,True,True,False
4,50,9.7,2,11.3,365,237.65,62,87.54,True,False,False,False,False,True


In [16]:
X = df_encoded.drop(columns=[predict_column])
y = df_encoded[predict_column]

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {X.columns.tolist()}")

Features shape: (80000, 13)
Target shape: (80000,)

Features: ['Age', 'Sleep_Duration', 'Stress_Level', 'Daily_Screen_Time', 'Caffeine_Intake', 'Reaction_Time', 'Memory_Test_Score', 'Gender_Male', 'Gender_Other', 'Diet_Type_Vegan', 'Diet_Type_Vegetarian', 'Exercise_Frequency_Low', 'Exercise_Frequency_Medium']


In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training: {X_train.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")

Training: 64000 samples
Test: 16000 samples


In [18]:
model = LinearRegression()
model.fit(X_train, y_train)

print("✅ Model is trained correctly")

✅ Model is trained correctly


In [19]:
y_pred = model.predict(X_test)

comparison_df = pd.DataFrame({
    'Real': y_test.values[:10],
    'Prediction': y_pred[:10],
    'Difference': abs(y_test.values[:10] - y_pred[:10])
})
comparison_df

,Real,Prediction,Difference
0,65.80,65.430017,0.369983
1,79.87,78.902513,0.967487
2,56.59,56.426283,0.163717
3,47.61,47.800048,0.190048
4,92.69,91.849655,0.840345
5,88.32,87.329451,0.990549
6,77.21,76.753286,0.456714
7,58.72,58.499858,0.220142
8,34.26,34.606389,0.346389
9,57.42,57.041106,0.378894


### Model Evaluation

**Main metrics:**
- **R² (Coefficient of Determination)**: Indicates the percentage of variance in the target variable explained by the model. Value between 0 and 1; closer to 1 = better.
- **RMSE (Root Mean Squared Error)**: Average error in the same units as the target variable. Lower = better.
- **MAE (Mean Absolute Error)**: Average absolute error. Lower = better.

In [20]:
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(root_mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print("=" * 50)
print("📊 MODEL EVALUATION METRICS")
print("=" * 50)
print(f"\n🎯 R² Score: {r2:.4f} ({r2*100:.2f}%)")
print(f"   → The model explains {r2*100:.2f}% of the variance")
print(f"\n📏 RMSE: {rmse:.4f}")
print(f"   → Average error of ±{rmse:.2f} points in Cognitive Score")
print(f"\n📐 MAE: {mae:.4f}")
print(f"   → Mean absolute error of {mae:.2f} points")
print("\n" + "=" * 50)

📊 MODEL EVALUATION METRICS

🎯 R² Score: 0.9956 (99.56%)
   → The model explains 99.56% of the variance

📏 RMSE: 1.2333
   → Average error of ±1.23 points in Cognitive Score

📐 MAE: 0.6262
   → Mean absolute error of 0.63 points



In [21]:
fig = px.scatter(
    x=y_test, 
    y=y_pred,
    labels={'x': 'Real Values', 'y': 'Predictions'},
    title='Predictions vs Real Values'
)

fig.add_shape(
    type='line',
    x0=y_test.min(), y0=y_test.min(),
    x1=y_test.max(), y1=y_test.max(),
    line=dict(color='red', dash='dash')
)
fig.update_layout(width=700, height=500)
fig.show()

In [22]:
coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print(f"Intercept (β₀): {model.intercept_:.4f}\n")
print("Model coefficients (sorted by importance):")
coef_df

Intercept (β₀): 108.9312

Model coefficients (sorted by importance):


,Feature,Coefficient
11,Exercise_Frequency_Low,-14.475661
12,Exercise_Frequency_Medium,-4.710560
2,Stress_Level,-1.931558
1,Sleep_Duration,1.927948
3,Daily_Screen_Time,-1.449972
6,Memory_Test_Score,0.484705
5,Reaction_Time,-0.163675
4,Caffeine_Intake,-0.019253
7,Gender_Male,-0.014554
10,Diet_Type_Vegetarian,0.014485


In [23]:
fig = px.bar(
    coef_df,
    x='Coefficient',
    y='Feature',
    orientation='h',
    title='Feature Importance (Model Coefficients)',
    color='Coefficient',
    color_continuous_scale='RdBu_r'
)
fig.update_layout(height=500, yaxis={'categoryorder': 'total ascending'})
fig.show()

In [24]:
residuals = y_test - y_pred

fig = px.histogram(
    residuals, 
    nbins=50,
    title='Residuals Distribution',
    labels={'value': 'Residual (Error)', 'count': 'Frequency'}
)
fig.add_vline(x=0, line_dash="dash", line_color="red")
fig.update_layout(showlegend=False)
fig.show()

print(f"Mean of residuals: {residuals.mean():.6f}")
print(f"Standard deviation: {residuals.std():.4f}")

Mean of residuals: 0.008812
Standard deviation: 1.5210


### Export and Save the Model

We save the trained model so we can use it with new data in the future.

In [25]:
import joblib
import os

models_path = "../models"
os.makedirs(models_path, exist_ok=True)

model_filename = f"{models_path}/linear_regression_cognitive_score.joblib"
joblib.dump(model, model_filename)

model_info = {
    'feature_columns': X.columns.tolist(),
    'target_column': predict_column,
    'categorical_columns': categorical_columns
}
joblib.dump(model_info, f"{models_path}/model_info.joblib")

print(f"✅ Model saved at: {model_filename}")
print(f"✅ Model info saved at: {models_path}/model_info.joblib")

✅ Model saved at: ../models/linear_regression_cognitive_score.joblib
✅ Model info saved at: ../models/model_info.joblib

✅ Model info saved at: ../models/model_info.joblib


### Data Format for New Predictions

To use the model with new data, you must send the information in this format:

In [26]:
# Show the expected data format
print("📋 DATA FORMAT FOR NEW PREDICTIONS")
print("=" * 60)
print("\n1️⃣ Required numerical columns:")
for col in numerical_columns:
    if col != predict_column:
        print(f"   - {col}")

print("\n2️⃣ Required categorical columns:")
for col in categorical_columns:
    values = df[col].unique().tolist()
    print(f"   - {col}: {values}")

print("\n" + "=" * 60)

📋 DATA FORMAT FOR NEW PREDICTIONS

1️⃣ Required numerical columns:
   - Age
   - Sleep_Duration
   - Stress_Level
   - Daily_Screen_Time
   - Caffeine_Intake
   - Reaction_Time
   - Memory_Test_Score

2️⃣ Required categorical columns:
   - Gender: ['Female', 'Male', 'Other']
   - Diet_Type: ['Non-Vegetarian', 'Vegetarian', 'Vegan']
   - Exercise_Frequency: ['Medium', 'High', 'Low']



In [27]:
loaded_model = joblib.load(f"{models_path}/linear_regression_cognitive_score.joblib")
model_info = joblib.load(f"{models_path}/model_info.joblib")

new_data = pd.DataFrame({
    'Age': [30, 45, 25],
    'Sleep_Duration': [7.5, 6.0, 8.0],
    'Stress_Level': [5, 8, 3],
    'Daily_Screen_Time': [6.0, 10.0, 4.0],
    'Caffeine_Intake': [150, 300, 50],
    'Reaction_Time': [350, 450, 280],
    'Memory_Test_Score': [75, 60, 85],
    'Gender': ['Male', 'Female', 'Other'],
    'Diet_Type': ['Vegetarian', 'Non-Vegetarian', 'Vegan'],
    'Exercise_Frequency': ['High', 'Low', 'Medium']
})

print("📊 New input data:")
new_data

📊 New input data:


,Age,Sleep_Duration,Stress_Level,Daily_Screen_Time,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Gender,Diet_Type,Exercise_Frequency
0,30,7.5,5,6.0,150,350,75,Male,Vegetarian,High
1,45,6.0,8,10.0,300,450,60,Female,Non-Vegetarian,Low
2,25,8.0,3,4.0,50,280,85,Other,Vegan,Medium


In [28]:
new_data_encoded = pd.get_dummies(
    new_data, 
    columns=model_info['categorical_columns'], 
    drop_first=True
)

for col in model_info['feature_columns']:
    if col not in new_data_encoded.columns:
        new_data_encoded[col] = False

new_data_encoded = new_data_encoded[model_info['feature_columns']]

print("📊 Processed data (One-Hot Encoded):")
new_data_encoded

📊 Processed data (One-Hot Encoded):


,Age,Sleep_Duration,Stress_Level,Daily_Screen_Time,Caffeine_Intake,Reaction_Time,Memory_Test_Score,Gender_Male,Gender_Other,Diet_Type_Vegan,Diet_Type_Vegetarian,Exercise_Frequency_Low,Exercise_Frequency_Medium
0,30,7.5,5,6.0,150,350,75,True,False,False,True,False,False
1,45,6.0,8,10.0,300,450,60,False,False,False,False,True,False
2,25,8.0,3,4.0,50,280,85,False,True,True,False,False,True


In [29]:
predictions = loaded_model.predict(new_data_encoded)

result = new_data.copy()
result['Predicted_Cognitive_Score'] = predictions.round(2)

print("🎯 COGNITIVE SCORE PREDICTIONS")
print("=" * 60)
result[['Age', 'Gender', 'Exercise_Frequency', 'Stress_Level', 'Predicted_Cognitive_Score']]

🎯 COGNITIVE SCORE PREDICTIONS


,Age,Gender,Exercise_Frequency,Stress_Level,Predicted_Cognitive_Score
0,30,Male,High,5,81.22
1,45,Female,Low,8,25.73
2,25,Other,Medium,3,102.46
